### TPC-DS Power Test

#### Add spark-sql-perf jar to Spark class path

Replace below with your cluster's spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar path

In [ ]:
%%init_spark
launcher.conf.set("spark.driver.extraClassPath", "/home/ubuntu/runtime/benchmark-tools/spark-sql-perf/target/scala-2.12/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar")
launcher.conf.set("spark.executor.extraClassPath", "/home/ubuntu/runtime/benchmark-tools/spark-sql-perf/target/scala-2.12/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar")

In [ ]:
val scaleFactor = "1"           // data scale 1GB
val iterations = 1              // how many times to run the whole set of queries.
val format = "parquet"          // support parquer or orc
// support s3a://s3_bucket, gs://gs_bucket
// wasbs://container@storage_account.blob.core.windows.net
// abfs://container@storage_account.dfs.core.windows.net
// hdfs:///user/livy
// replace fsdir with your file system and bucket 
val fsdir = "s3a://s3_bucket_name"
val partitionTables = true      // create partition tables
val query_filter = Seq()        // Seq() == all queries
// val query_filter = Seq("q1-v2.4", "q2-v2.4") // run subset of queries
val randomizeQueries = false    // run queries in a random order. Recommended for parallel runs.


In [ ]:
// detailed results will be written as JSON to this location.
val data_path= s"${fsdir}/shared/data/tpcds/tpcds_${format}/${scaleFactor}"
var resultLocation = s"${fsdir}/shared/data/results/tpcds_${format}/${scaleFactor}/"
var databaseName = s"tpcds_${format}_scale_${scaleFactor}_db"
val use_arrow = false            // when you want to use gazella_plugin to run TPC-DS, you need to set it true.

if (use_arrow){
    resultLocation = s"${fsdir}/shared/data/results/tpcds_arrow/${scaleFactor}/"
    val tables = Seq("call_center", "catalog_page", "catalog_returns", "catalog_sales", "customer", "customer_address", "customer_demographics", "date_dim", "household_demographics", "income_band", "inventory", "item", "promotion", "reason", "ship_mode", "store", "store_returns", "store_sales", "time_dim", "warehouse", "web_page", "web_returns", "web_sales", "web_site")
    if (spark.catalog.databaseExists(s"$databaseName")) {
        println(s"$databaseName has exists!")
    }else{
        spark.sql(s"create database if not exists $databaseName").show
        spark.sql(s"use $databaseName").show
        for (table <- tables) {
            if (spark.catalog.tableExists(s"$table")){
                println(s"$table has exists!")
            }else{
                spark.catalog.createTable(s"$table", s"$data_path/$table", "arrow")
            }
        }
        if (partitionTables) {
            for (table <- tables) {
                try{
                    spark.sql(s"ALTER TABLE $table RECOVER PARTITIONS").show
                }catch{
                        case e: Exception => println(e)
                }
            }
        }
    }
} else {
    // Check whether the database is created, we create external tables if not
    if (spark.catalog.databaseExists(s"$databaseName")) {
        println(s"Using existing $databaseName")
    } else {
        import com.databricks.spark.sql.perf.tpcds.TPCDSTables

        println(s"$databaseName doesn't exist. Creating...")
        val tables = new TPCDSTables(spark.sqlContext, "", s"${scaleFactor}", false)
        tables.createExternalTables(data_path, format, databaseName, overwrite = true, discoverPartitions = partitionTables)
    }
}

val timeout = 60 // timeout in hours


In [ ]:
// COMMAND ----------
// Spark configuration
spark.conf.set("spark.sql.broadcastTimeout", "10000") // good idea for Q14, Q88.

// ... + any other configuration tuning
// COMMAND ----------


In [ ]:
sql(s"use $databaseName")
import com.databricks.spark.sql.perf.tpcds.TPCDS

val tpcds = new TPCDS (sqlContext = spark.sqlContext)
def queries = {
  val filtered_queries = query_filter match {
    case Seq() => tpcds.tpcds2_4Queries
    case _ => tpcds.tpcds2_4Queries.filter(q => query_filter.contains(q.name))
  }
  if (randomizeQueries) scala.util.Random.shuffle(filtered_queries) else filtered_queries
}
val experiment = tpcds.runExperiment(
  queries,
  iterations = iterations,
  resultLocation = resultLocation,
  tags = Map("runtype" -> "benchmark", "database" -> databaseName, "scale_factor" -> scaleFactor))

println(experiment.toString)
experiment.waitForFinish(timeout*60*60)

val res=experiment.getCurrentResults // or: spark.read.json(resultLocation).filter("timestamp = 1429132621024")
  .withColumn("Name", substring(col("name"), 1, 100))
  .withColumn("Runtime_sec", round(((col("parsingTime") + col("analysisTime") + col("optimizationTime") + col("planningTime") + col("executionTime")) / 1000.0), 2))
  .select('Name, 'Runtime_sec)
res.show(104)

val sumRuntime: Double = res.agg(sum("Runtime_sec").cast("double")).first.getDouble(0)
println(s"Total time: ${sumRuntime}s")
